In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
from textblob import TextBlob


In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt

In [3]:
import warnings
#warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

In [225]:
from langdetect import detect

def language_detection(text):
    try:
        return detect(text)
    except:
        return None

In [4]:
import spacy
from spacy import displacy
import collections

# nlp = spacy.load("en_core_web_trf", disable=['tok2vec', 'lemmatizer', 'ner']) # accuracy
#nlp = spacy.load('en_core_web_sm', disable=['tok2vec', 'lemmatizer', 'ner']) # efficiency
nlp = spacy.load('en_core_web_sm', disable = ['lemmatizer'])

/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/spacy/util.py:1176: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/en_core_web_sm/en_core_web_sm-3.0.0/tok2vec/model'>
  reader(path / key)
/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/spacy/util.py:1176: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/en_core_web_sm/en_core_web_sm-3.0.0/tagger/model'>
  reader(path / key)
/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/spacy/util.py:1176: ResourceWarning: unclosed file <_io.BufferedReader name='/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/en_core_web_sm/en_core_web_sm-3.0.0/senter/model'>
  reader(path / key)


date range

In [227]:
# date range
start_date = "2019-01-01"
end_date = "2021-12-31"

define keywords: Singapore

In [344]:
## aspect
transport = ['location', 'bus', 'buses', 'busstop', 'mrt', 'station', 'stations', 'walk', 
            'attraction', 'attractions', 
            'center', 'centers', 'centre', 'centres', 'connections', 'connection', 'connectivity', 'court', 
            'direction', 'directions', 'ferry', 
            'inconvenience', 'inconvenient', 'convenience', 'convenient',
            'min', 'mins', 'minute', 'minutes', 'parking', 'proximity', 'ride', 
            'stop', 'stops', 'train', 'trains', 
            'transport', 'transportation', 'transports', 'mrts', 'tube', 'tubes', 'commuting', 'walking'
            'accessible', 'convenient', 'inconvenient', 'close', 'far', 'walkable', 'near', 'central',
            'connected', 'underground', 'undergrounds'
            ]

amenities = ['amenities', 'cafe', 'coffee', 'club', 'clubs', 'eateries', 'food', 'foods', 'foodcourt', 
            'fruit', 'grocery', 'groceries', 
            'hawker', 'hawkers', 'mall', 'malls', 'market', 'markets', 'restaurant', 'restaurants', 
            'shopping', 'shops', 'store', 'stores',
            'supermarket','eats', 'courts', 'foodcourts', 'bar', 'bars', 'mart', 'marts',
            'nightlife', 'plaza', 'plazas', 'coffeeshop', 'coffeeshops',
            'commercial', 'reachable', 'eatery',
            'theater', 'theatre', 'cinema', 'coffeehouse', 'coffeehouses', 'coffeeplaces'
            ]

greenery = ['garden', 'gardens', 'greeneries', 'greenery', 'park', 'green', 'tree']

safety = ['safety', 'security', 'dark', 'safe', 'unsafe', 'dangerous']

noise = ['noise', 'noises', 'noisy']


## description
transport_des = ['accessible', 'convenient', 'inconvenient', 'close', 'far', 'walkable', 'near', 
                'central','connected', 'walking']

amenities_des = ['commercial', 'reachable']

greenery_des = ['green']

safety_des = ['dark', 'safe', 'unsafe', 'dangerous']

noise_des = ['noisy']

import dataset

In [228]:
# import airbnb dataset

df = pd.read_csv('/Users/jiaxuan/OneDrive - National University of Singapore/MUP_year2/dissertation/datasets/singapore/reviews.csv')

# filter 2019-2021 (3 years) [df_fl]
after_start_date = df["date"] >= start_date
before_end_date = df["date"] <= end_date
between_two_dates = after_start_date & before_end_date

df_fl = df.loc[between_two_dates]

# checking shape
print("The filtered dataset has {} rows and {} columns.".format(*df_fl.shape))

# drop na
df_fl.dropna(inplace=True)

# create column language
df_fl['language'] = df_fl['comments'].apply(language_detection)
df_fl_en = df_fl[(df_fl['language']=='en')]

# remove '<br/>'
df_fl_en['comments'] = df_fl_en['comments'].str.replace('<br/>', ' ', regex = False)

The filtered dataset has 25724 rows and 6 columns.


/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [229]:
# sentencize for aspect_des
df_fl_en['comments'] = df_fl_en["comments"].apply(lambda x: [sent.text for sent in nlp(x).sents])

/Users/jiaxuan/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [234]:
# assign each sentence a sentence id
df_fl_en = df_fl_en.explode("comments", ignore_index=True)
df_fl_en.index.name = "Sentence ID"
df_fl_en = df_fl_en.reset_index()

# further remove non-en sentences (some comments contains multiple languages)
df_fl_en['language'] = df_fl_en['comments'].apply(language_detection)
df_fl_en = df_fl_en[(df_fl_en['language']=='en')]

The reasons why I still explode the comments into sentences are:
1. I want to get a more accurate sentiment score on each aspect, such as individual score on transportation, indiv score on greenery... But if I don't break the comment into sentences, the outcome of [overall_sentiment] will be less accurate regarding the one specific aspect. In that case, only polarity of the descriptive word will work.
2. To be honest I don't know how and don't want to put a sentencizer into that [aspect_des] function... ;_;

In [5]:
def aspect_description(text):
    doc = nlp(text)
    aspect_des = []
    ####description = []

    for token in doc:
        # root = adj, extract root
        if token.pos_ == "ADJ":
            if token.head.pos_ == "NOUN":
                aspect_des.append((str(token.head.text).lower(), str(token.text).lower()))
            for child in token.children:
                if token.head.pos_ == "NOUN" and child.pos_ == "ADJ" and child.dep_ == "conj":
                    aspect_des.append((str(token.head.text).lower(), str(child.text).lower()))
                elif token.head.pos_ == "NOUN" and child.pos_ == "ADJ" and child.dep_ == "amod":
                    #aspect_des.remove((str(token.head.text).lower(), str(token.text).lower()))
                    aspect_des.append((str(token.head.text).lower(), str(child.text).lower() + " " + str(token.text).lower()))    
        
        # root = noun, extract root 
        elif token.pos_ in ("NOUN", "PRON") and token.head.pos_ == "AUX":
            for child in token.head.children:
                if child.pos_ == "ADJ" and token.head.pos_ in ("NOUN", "PRON"):
                    aspect_des.append((str(token.text).lower(), str(child.text).lower()))
                    for gchild in child.children:
                        if gchild.pos_ == "ADJ" and gchild.dep_ == "conj":
                            aspect_des.append((str(token.text).lower(), str(gchild.text).lower()))
                    
        # root = verb, extract nominal subject and direct object, depending on cases
        elif token.pos_ == "VERB":
            for child in token.children:
                if child.pos_ == "NOUN" and child.dep_ == "dobj" and token.head.pos_ == "VERB": 
                    aspect_des.append((str(child.text).lower(), str(token.head.text).lower()))
                elif token.dep_ == "conj" and child.pos_ == "ADJ" and child.dep_ == "acomp":
                    for gchild in token.children:
                        if gchild.pos_ in ("NOUN", "PRON") and gchild.dep_ == "nsubj":
                            aspect_des.append((str(gchild.text).lower(), str(child.text).lower())) 
        
        else: 
            continue        
    return aspect_des


In [12]:
# test
text = 'Everything was just nice, and the location is excellent.'
doc = nlp(text)
for token in doc:
    print(token.text, token.dep_,'; ', token.head.text, token.head.pos_,
            [child for child in token.children])

print(aspect_description(text))
displacy.render(doc, style="dep")

Everything nsubj ;  was AUX []
was ROOT ;  was AUX [Everything, nice, ,, and, is]
just advmod ;  nice ADJ []
nice acomp ;  was AUX [just]
, punct ;  was AUX []
and cc ;  was AUX []
the det ;  location NOUN []
location nsubj ;  is VERB [the]
is conj ;  was AUX [location, excellent, .]
excellent acomp ;  is VERB []
. punct ;  is VERB []
[('location', 'excellent')]


In [10]:
# test
text = 'The irritating host cancelled our reservation only 1 day before and now the platform is indifferent.'
doc = nlp(text)
for token in doc:
    print(token.text, token.dep_,'; ', token.head.text, token.head.pos_,
            [child for child in token.children])

print(aspect_description(text))
displacy.render(doc, style="dep")

The det ;  host NOUN []
irritating amod ;  host NOUN []
host nsubj ;  cancelled VERB [The, irritating]
cancelled ROOT ;  cancelled VERB [host, reservation, before, and, is]
our poss ;  reservation NOUN []
reservation dobj ;  cancelled VERB [our]
only advmod ;  1 NUM []
1 nummod ;  day NOUN [only]
day npadvmod ;  before ADV [1]
before advmod ;  cancelled VERB [day]
and cc ;  cancelled VERB []
now advmod ;  is VERB []
the det ;  platform NOUN []
platform nsubj ;  is VERB [the]
is conj ;  cancelled VERB [now, platform, indifferent, .]
indifferent acomp ;  is VERB []
. punct ;  is VERB []
[('host', 'irritating'), ('reservation', 'cancelled'), ('platform', 'indifferent')]


In [8]:
# test
text = 'Very nice little place'
doc = nlp(text)
for token in doc:
    print(token.text, token.dep_,'; ', token.head.text, token.head.pos_,
            [child for child in token.children])

print(aspect_description(text))
displacy.render(doc, style="dep")

Very advmod ;  nice ADJ []
nice amod ;  place NOUN [Very]
little amod ;  place NOUN []
place ROOT ;  place NOUN [nice, little]
[('place', 'nice'), ('place', 'little')]


In [231]:
# apply func(polarity) to a sentence => overall_sentiment, to a descriptive word => polarity
def polarity(sentence):
    score = TextBlob(sentence).sentiment.polarity
    return score

# apply func(polarity) to a sentence => overall_sentiment, to a descriptive word => polarity
def subjectivity(sentence):
    score = TextBlob(sentence).sentiment.subjectivity
    return score

In [278]:
df_fl_en['aspect_pair'] = df_fl_en['comments'].apply(aspect_description)

In [279]:
df_fl_en['aspect_pair']

0               [(reservation, canceled)]
1                                      []
2                   [(place, attractive)]
3                                      []
4                                      []
                       ...               
74151    [(place, nice), (place, little)]
74152                      [(play, nice)]
74153                                  []
74154                                  []
74155                    [(place, great)]
Name: aspect_pair, Length: 66921, dtype: object

In [284]:
df_fl_en_notnull = df_fl_en[df_fl_en['aspect_pair'].map(lambda d: len(d)) > 0]
#df_fl_en_notnull
#df_fl_en_notnull.to_csv("/Users/jiaxuan/Desktop/2.csv", index = False)

,Sentence ID,listing_id,id,date,reviewer_id,reviewer_name,comments,language,aspect_pair
0,0,71609,368048609,2019-01-06,96913513,Yasukawa,The host canceled this reservation 123 days be...,en,"[(reservation, canceled)]"
2,2,71609,496082140,2019-07-27,220147487,John,Attractive place to stay for 4pax when visit S...,en,"[(place, attractive)]"
10,10,71609,573744808,2019-12-07,312920122,Jean-Philippe,and she hace us some a lot of things to see an...,en,"[(lot, hace)]"
12,12,71609,581140325,2019-12-24,96914254,Jade,"Great location, great hospitality",en,"[(location, great), (hospitality, great)]"
14,14,71609,593817449,2020-01-17,29197511,Elton,Belinda was really welcoming and the checking ...,en,"[(checking, easy)]"
...,...,...,...,...,...,...,...,...,...
74148,74148,50404949,405347261484316469,2021-07-13,237160928,Aravin,"When cleaning room, no new bottled water was g...",en,"[(water, new), (water, bottled)]"
74150,74150,50404949,405347261484316469,2021-07-13,237160928,Aravin,There was no WiFi connection for several days.,en,"[(days, several)]"
74151,74151,50492656,397911660811744659,2021-07-02,391334830,Bugao,Very nice little place,en,"[(place, nice), (place, little)]"
74152,74152,50542602,398895510974206210,2021-07-04,10655446,Celinie,A nice play to stay.,en,"[(play, nice)]"


In [287]:
df_fl_en_notnull = df_fl_en_notnull.explode("aspect_pair", ignore_index=True)
df_fl_en_notnull.index.name = "aspect ID"
df_fl_en_notnull = df_fl_en_notnull.reset_index()

In [292]:
tempdf = df_fl_en_notnull["aspect_pair"].apply(pd.Series)
df_fl_en_notnull["aspect"] = tempdf[0]
df_fl_en_notnull["description"] = tempdf[1]

In [294]:
df_fl_en_notnull['overall_sentiment'] = df_fl_en_notnull['comments'].apply(polarity)
df_fl_en_notnull['overall_subjectivity'] = df_fl_en_notnull['comments'].apply(subjectivity)
df_fl_en_notnull['des_polarity'] = df_fl_en_notnull['description'].apply(polarity)
df_fl_en_notnull['des_subjectivity'] = df_fl_en_notnull['description'].apply(subjectivity)

In [296]:
#df_fl_en_notnull.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/final_aspect_results/aspect_sent_output.csv", index = False)

In [339]:
#pd.DataFrame(df_fl_en_notnull['aspect'].unique()).to_csv('/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/final_aspect_results/aspect_keywords.csv')
#pd.DataFrame(df_fl_en_notnull['description'].unique()).to_csv('/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/final_aspect_results/description_keywords.csv')


join coordinates df

In [ ]:
listings = pd.read_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/listings.csv")
listings.rename(columns={'id':'listing_id'}, inplace = True)
listings_coord = listings[['listing_id', 'latitude','longitude']]
df_aspect_coord = pd.merge(df_fl_en_notnull, listings_coord, on = ['listing_id'])

generate geometry

In [307]:
#gdf_aspect = gpd.GeoDataFrame(df_aspect_coord, geometry=gpd.points_from_xy(df_aspect_coord.longitude, df_aspect_coord.latitude))

In [ ]:
#gdf_aspect.head(5)

In [310]:
#gdf_aspect.to_csv("/Users/jiaxuan/Desktop/3.csv", index = False)

group by aspects

In [345]:
df_transport = df_aspect_coord[df_aspect_coord['aspect'].isin(transport)]
df_amenities = df_aspect_coord[df_aspect_coord['aspect'].isin(amenities)]
df_greenery = df_aspect_coord[df_aspect_coord['aspect'].isin(greenery)]
df_safety = df_aspect_coord[df_aspect_coord['aspect'].isin(safety)]
df_noise = df_aspect_coord[df_aspect_coord['aspect'].isin(noise)]

df_transport_des = df_aspect_coord[df_aspect_coord['description'].isin(transport_des)] 
df_amenities_des = df_aspect_coord[df_aspect_coord['description'].isin(amenities_des)]
df_greenery_des = df_aspect_coord[df_aspect_coord['description'].isin(greenery_des)]
df_safety_des = df_aspect_coord[df_aspect_coord['description'].isin(safety_des)]
df_noise_des = df_aspect_coord[df_aspect_coord['description'].isin(noise_des)]

In [342]:
df_transport_final = pd.concat([df_transport,df_transport_des]).drop_duplicates()
df_amenities_final = pd.concat([df_amenities,df_amenities_des]).drop_duplicates()
df_greenery_final = pd.concat([df_greenery,df_greenery_des]).drop_duplicates()
df_safety_final = pd.concat([df_safety,df_safety_des]).drop_duplicates()
df_noise_final = pd.concat([df_noise,df_noise_des]).drop_duplicates()

In [346]:
df_transport_final.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/transport.csv", index = False)
df_amenities_final.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/amenities.csv", index = False)
df_greenery_final.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/greenery.csv", index = False)
df_safety_final.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/safety.csv", index = False)
df_noise_final.to_csv("/Users/jiaxuan/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/MUP_year2/dissertation/datasets/singapore/qgis/noise.csv", index = False)
